In [1]:
import cv2
import mediapipe as mp 
import math

In [2]:
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

In [3]:
def distancia(p1, p2):
    return math.sqrt((p2.x - p1.x)**2 + (p2.y - p1.y)**2 + (p2.z - p1.z)**2)

In [20]:
# Función para determinar si un dedo está levantado 
def dedo_levantado(landmarks):
    dedos = {
        'Pulgar': False,
        'Indice': False,
        'Medio': False,
        'Anular': False,
        'Menique': False
    }
    # Pulgar (comparacion en eje X para mejor precision)
    dedos['Pulgar'] = landmarks[4].x < landmarks[3].x if landmarks[0].x < landmarks[9].x else landmarks[4].x > landmarks[3].x
    dedos['Indice'] = landmarks[8].y < landmarks[6].y
    dedos['Medio'] = landmarks[12].y < landmarks[10].y
    dedos['Anular'] = landmarks[16].y < landmarks[14].y
    dedos['Menique'] = landmarks[20].y < landmarks[18].y
    
    return dedos

def detectar_gesto(dedos):
    # Gestos predefinidos
    if all (dedos.values()):
        return 'Alto'
    elif dedos['Indice'] and dedos['Menique'] and not any([dedos['Pulgar'], dedos['Anular'], dedos['Medio']]):
        return 'Cuernos'
    elif dedos['Pulgar'] and dedos['Menique'] and not any([dedos['Indice'], dedos['Medio'], dedos['Anular']]):
        return 'Llamame'
    elif dedos['Indice'] and dedos['Menique'] and dedos['Pulgar'] and not any([dedos['Anular'], dedos['Medio']]):
        return 'Te quiero '
    elif dedos['Indice'] and dedos['Menique'] and dedos['Pulgar'] and not any([dedos['Anular'], dedos['Medio']]):
        return 'Saludo vulcano'
    elif not any(dedos.values()):
        return 'Puño'
    elif any([dedos['Medio'], dedos['Anular'], dedos['Menique']]) and distancia(landmarks[4], landmarks[8]) < 0.05:
        return 'Pinza'
    elif any([dedos['Medio'], dedos['Anular'], dedos['Menique']]) and distancia(landmarks[4], landmarks[8]) < 0.05:
        return 'Senialar izquierda'
    elif any([dedos['Medio'], dedos['Anular'], dedos['Menique']]) and distancia(landmarks[4], landmarks[8]) < 0.05:
        return 'Senialar derecha'
    elif any([dedos['Medio'], dedos['Anular'], dedos['Menique']]) and distancia(landmarks[4], landmarks[8]) < 0.05:
        return 'Corazon coreano'
    else:
        return 'Gesto no definido'

In [21]:
cap = cv2.VideoCapture(0) # 0 camara predeterminado

In [22]:
with mp_hands.Hands(
    static_image_mode = False,
    max_num_hands = 2,
    min_detection_confidence = 0.7,
    min_tracking_confidence=0.7) as hands:
    
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            print('No se pudo capturar imagen')
            continue
        image = cv2.flip(image, 1)
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = hands.process(image_rgb)

        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(
                    image, hand_landmarks, mp_hands.HAND_CONNECTIONS
                )
                landmarks = hand_landmarks.landmark


                dedos = dedo_levantado(landmarks)
                gesto = detectar_gesto(dedos)

                # Mostrar gesto detectado
                cv2.putText(image, f'Gesto: {gesto}', (10,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

                info_dedos = f'Dedos: {[k for k, v in dedos.items() if v]}'

                cv2.putText(image, info_dedos, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 0), 2)
                
        cv2.imshow('Deteccion de gestos', image)

        if cv2.waitKey(5) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()